In [1]:
import pandas as pd
import numpy as np

In [11]:
problems = pd.read_json('./problems.json')

In [12]:
problems.head()

,id,contest_id,title
0,abc001_1,abc001,A. 積雪深差
1,abc001_2,abc001,B. 視程の通報
2,abc001_3,abc001,C. 風力観測
3,abc001_4,abc001,D. 感雨時刻の整理
4,abc002_1,abc002,A. 正直者


In [13]:
problems.to_csv('./problems.csv', header=False, index=False)

In [14]:
contests = problems['contest_id'].unique()

In [15]:
types = []
for contest in contests:
    initial = contest[:3]
    contest_type  = ''
    if initial == 'abc':
        contest_type = 1
    elif initial == 'arc':
        contest_type = 2
    elif initial == 'agc':
        contest_type = 3
    else:
        contest_type = 4
    
    types.append(contest_type)

In [18]:
contests_data.to_csv('./contests.csv', header=False, index=False)

In [16]:
contests_data = pd.DataFrame({'id': contests, 'type': types})

In [17]:
contests_data.head()

,id,type
0,abc001,1
1,abc002,1
2,abc003,1
3,abc004,1
4,abc005,1


In [2]:
need_columns = ['user_id', 'problem_id', 'language', 'execution_time', 'length', 'result']
data = pd.read_csv('./submissions.csv', usecols=need_columns)
data = data.dropna()
data = data.astype({'execution_time': 'int32'})

In [4]:
# scale down for local test
# data = data.iloc[:100000, :]
data_ac = data[data['result'] == 'AC']

In [5]:
list_for_exec = ['problem_id', 'language', 'execution_time']
list_for_leng =  ['problem_id', 'language', 'length']
data_exec = data_ac[list_for_exec]
data_leng = data_ac[list_for_leng]

In [6]:
group_prob_lang_exec = data_exec.groupby(['language', 'problem_id'], as_index=False)

In [7]:
exec_statistics = group_prob_lang_exec.quantile(0.1, interpolation='higher').rename(columns={'execution_time': '10%'})
exec_statistics['25%'] = group_prob_lang_exec.quantile(0.25, interpolation='higher')['execution_time']
exec_statistics['50%'] =  group_prob_lang_exec.quantile(0.50, interpolation='higher')['execution_time']
exec_statistics['75%'] =  group_prob_lang_exec.quantile(0.75, interpolation='higher')['execution_time']

In [38]:
exec_statistics.to_csv('./exec_statistics', header=False)

In [8]:
exec_statistics.head()

,language,problem_id,10%,25%,50%,75%
0,Ada2012 (GNAT 9.2.1),abc162_a,4,4,4,4
1,Ada2012 (GNAT 9.2.1),abc164_a,5,5,5,5
2,Ada2012 (GNAT 9.2.1),abc164_b,3,3,3,3
3,Ada2012 (GNAT 9.2.1),abc165_a,4,4,4,4
4,Awk (GNU Awk 4.1.4),abc162_a,3,4,8,12


In [9]:
group_prob_lang_leng = data_leng.groupby(['language', 'problem_id'], as_index=False)

In [10]:
length_statistics = group_prob_lang_leng.quantile(0.1, interpolation='higher').rename(columns={'length': '10%'})
length_statistics['25%'] = group_prob_lang_leng.quantile(0.25, interpolation='higher')['length']
length_statistics['50%'] =  group_prob_lang_leng.quantile(0.50, interpolation='higher')['length']
length_statistics['75%'] =  group_prob_lang_leng.quantile(0.75, interpolation='higher')['length']

In [11]:
length_statistics.head()

,language,problem_id,10%,25%,50%,75%
0,Ada2012 (GNAT 9.2.1),abc162_a,438,438,438,438
1,Ada2012 (GNAT 9.2.1),abc164_a,299,299,299,299
2,Ada2012 (GNAT 9.2.1),abc164_b,483,483,483,483
3,Ada2012 (GNAT 9.2.1),abc165_a,423,423,423,423
4,Awk (GNU Awk 4.1.4),abc162_a,17,17,19,20


In [22]:
length_statistics.drop('min', axis=1).to_csv('./length_statistics', header=False)

In [12]:
length_statistics['keys'] = length_statistics['language'] + length_statistics['problem_id']

In [13]:
length_statistics.head()

,language,problem_id,10%,25%,50%,75%,keys
0,Ada2012 (GNAT 9.2.1),abc162_a,438,438,438,438,Ada2012 (GNAT 9.2.1)abc162_a
1,Ada2012 (GNAT 9.2.1),abc164_a,299,299,299,299,Ada2012 (GNAT 9.2.1)abc164_a
2,Ada2012 (GNAT 9.2.1),abc164_b,483,483,483,483,Ada2012 (GNAT 9.2.1)abc164_b
3,Ada2012 (GNAT 9.2.1),abc165_a,423,423,423,423,Ada2012 (GNAT 9.2.1)abc165_a
4,Awk (GNU Awk 4.1.4),abc162_a,17,17,19,20,Awk (GNU Awk 4.1.4)abc162_a


In [14]:
length_border_dict = length_statistics.set_index('keys').drop(['language', 'problem_id'], axis=1).to_dict(orient='index')

In [15]:
exec_statistics['keys'] = exec_statistics['language'] + exec_statistics['problem_id']

In [16]:
exec_border_dict = exec_statistics.set_index('keys').drop(['language', 'problem_id'], axis=1).to_dict(orient='index')

In [17]:
def get_score_by_border(value, border):
    if value <= border['10%']:
        return 5
    
    elif value <= border['25%']:
        return 4
    
    elif value <= border['50%']:
        return 3
    
    elif value <= border['75%']:
        return 2
    
    else:
        return 1

In [18]:
list_for_user_info = ['user_id', 'problem_id', 'language', 'length', 'execution_time']
pre_user_info = data_ac[list_for_user_info]

In [19]:
pre_user_info.head()

,user_id,problem_id,language,length,execution_time
0,i_lohas_MATCH,abc151_c,C++14 (GCC 5.4.1),777,73
1,rniya,abc141_b,C++14 (GCC 5.4.1),1002,1
2,naoki2016,abc067_b,C++14 (GCC 5.4.1),598,1
4,t_kato,dwango2017qual_a,PyPy3 (2.4.0),99,172
5,zoooma13,agc006_b,C++14 (GCC 5.4.1),651,38


In [20]:
group_user_lang_prob = pre_user_info.groupby(['user_id', 'language', 'problem_id'], as_index=False)

In [21]:
user_info = group_user_lang_prob.min()

In [22]:
user_info.size

27562045

In [23]:
user_info.head()

,user_id,language,problem_id,length,execution_time
0,0071717,Python3 (3.4.3),abc161_b,704,17
1,011337,C++ (GCC 9.2.1),abc162_a,203,2
2,011337,C++ (GCC 9.2.1),abc162_b,204,7
3,011337,C++ (GCC 9.2.1),abc162_c,386,346
4,011337,C++ (GCC 9.2.1),abc164_a,154,7


In [24]:
user_info['keys'] = user_info['language'] + user_info['problem_id']

In [25]:
user_info['keys_length'] = user_info['keys'] + '&' + user_info['length'].astype('str')
user_info['keys_exec'] = user_info['keys'] + '&' + user_info['execution_time'].astype('str')

In [26]:
user_info.head()

,user_id,language,problem_id,length,execution_time,keys,keys_length,keys_exec
0,0071717,Python3 (3.4.3),abc161_b,704,17,Python3 (3.4.3)abc161_b,Python3 (3.4.3)abc161_b&704,Python3 (3.4.3)abc161_b&17
1,011337,C++ (GCC 9.2.1),abc162_a,203,2,C++ (GCC 9.2.1)abc162_a,C++ (GCC 9.2.1)abc162_a&203,C++ (GCC 9.2.1)abc162_a&2
2,011337,C++ (GCC 9.2.1),abc162_b,204,7,C++ (GCC 9.2.1)abc162_b,C++ (GCC 9.2.1)abc162_b&204,C++ (GCC 9.2.1)abc162_b&7
3,011337,C++ (GCC 9.2.1),abc162_c,386,346,C++ (GCC 9.2.1)abc162_c,C++ (GCC 9.2.1)abc162_c&386,C++ (GCC 9.2.1)abc162_c&346
4,011337,C++ (GCC 9.2.1),abc164_a,154,7,C++ (GCC 9.2.1)abc164_a,C++ (GCC 9.2.1)abc164_a&154,C++ (GCC 9.2.1)abc164_a&7


In [27]:
def create_calcu_func(border_dict):
    def calcurate_score(key_value):
        key, value = key_value.split('&')
        value = int(value)
        border = border_dict[key]
        score = get_score_by_border(value, border)
        return score
    return calcurate_score

In [28]:
calcu_exec_score = create_calcu_func(exec_border_dict)
calcu_length_score = create_calcu_func(length_border_dict)

In [29]:
length_scores = user_info['keys_length'].map(calcu_length_score)
exec_scores = user_info['keys_exec'].map(calcu_exec_score)

In [30]:
user_info['length_score'] = length_scores

In [31]:
user_info['exec_score'] = exec_scores

In [32]:
user_info.head()

,user_id,language,problem_id,length,execution_time,keys,keys_length,keys_exec,length_score,exec_score
0,0071717,Python3 (3.4.3),abc161_b,704,17,Python3 (3.4.3)abc161_b,Python3 (3.4.3)abc161_b&704,Python3 (3.4.3)abc161_b&17,1,5
1,011337,C++ (GCC 9.2.1),abc162_a,203,2,C++ (GCC 9.2.1)abc162_a,C++ (GCC 9.2.1)abc162_a&203,C++ (GCC 9.2.1)abc162_a&2,5,5
2,011337,C++ (GCC 9.2.1),abc162_b,204,7,C++ (GCC 9.2.1)abc162_b,C++ (GCC 9.2.1)abc162_b&204,C++ (GCC 9.2.1)abc162_b&7,5,2
3,011337,C++ (GCC 9.2.1),abc162_c,386,346,C++ (GCC 9.2.1)abc162_c,C++ (GCC 9.2.1)abc162_c&386,C++ (GCC 9.2.1)abc162_c&346,4,2
4,011337,C++ (GCC 9.2.1),abc164_a,154,7,C++ (GCC 9.2.1)abc164_a,C++ (GCC 9.2.1)abc164_a&154,C++ (GCC 9.2.1)abc164_a&7,5,1


In [33]:
user_status = user_info.loc[:, ['user_id', 'language', 'length_score', 'exec_score']]

In [34]:
user_status.head()

,user_id,language,length_score,exec_score
0,0071717,Python3 (3.4.3),1,5
1,011337,C++ (GCC 9.2.1),5,5
2,011337,C++ (GCC 9.2.1),5,2
3,011337,C++ (GCC 9.2.1),4,2
4,011337,C++ (GCC 9.2.1),5,1


In [35]:
user_scores = user_status.groupby(['user_id', 'language'], as_index=False).sum()
user_ac_count = user_status.groupby(['user_id', 'language'], as_index=False).count()

In [36]:
user_scores['ac_count'] = user_ac_count['length_score'].values

In [37]:
user_scores['length_ave'] = user_scores['length_score'] / user_scores['ac_count']
user_scores['exec_ave'] = user_scores['exec_score'] / user_scores['ac_count']

In [38]:
user_scores.head()

,user_id,language,length_score,exec_score,ac_count,length_ave,exec_ave
0,0071717,Python3 (3.4.3),1,5,1,1.000000,5.000000
1,011337,C++ (GCC 9.2.1),34,22,7,4.857143,3.142857
2,011337,C++14 (GCC 5.4.1),34,36,8,4.250000,4.500000
3,04089,C (GCC 9.2.1),5,3,2,2.500000,1.500000
4,04089,C++ (GCC 9.2.1),9,4,2,4.500000,2.000000


In [44]:
user_scores.drop(['length_ave', 'exec_ave'], axis=1).to_csv('./user_rankings.csv', index=False, header=False)